In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, row_number, floor, collect_list
from pyspark.sql.window import Window
from pyspark.sql.functions import regexp_extract, length

# 1. SparkSession
spark = SparkSession.builder.appName("FASTQ_Validation_Efficient").getOrCreate()

# 2. Ścieżki do FASTQ
fastq_dir = "/Volumes/databrics_2/default/fastq"
files = ["SRR16356246_1.fastq", "SRR16356246_2.fastq"]
paths = [f"{fastq_dir}/{file}" for file in files]

# 3. Wczytanie plików jako tekst
raw_df = spark.read.text(paths)

# 4. Numerowanie wierszy
window = Window.orderBy("value")  # bez partitionBy → ostrzeżenie jeśli duży plik
df_indexed = raw_df.withColumn("row_num", row_number().over(window))

# 5. Grupowanie co 4 linie
df_indexed = df_indexed.withColumn("group_id", floor((col("row_num")-1)/4))

fastq_grouped = df_indexed.groupBy("group_id") \
    .agg(collect_list("value").alias("lines"))

# 6. Mapowanie na kolumny FASTQ
fastq_df = fastq_grouped.select(
    col("lines").getItem(0).alias("header"),
    col("lines").getItem(1).alias("sequence"),
    col("lines").getItem(2).alias("plus"),
    col("lines").getItem(3).alias("quality")
)

fastq_df.show(5, truncate=False)

# 7. Walidacja długości
df_with_lengths = fastq_df.withColumn(
    "declared_length",
    regexp_extract(col("header"), r"length=(\d+)", 1).cast("int")
).withColumn(
    "actual_length",
    length(col("sequence"))
)

invalid_records = df_with_lengths.filter(
    col("declared_length") != col("actual_length")
)

invalid_count = invalid_records.count()
print("Liczba niespójnych rekordów:", invalid_count)

# =====================================================
# KOMENTARZE ANALITYCZNE SPARK UI
# =====================================================

# 1. Liczba Jobów = 1 (wywołanie .count() uruchamia wykonanie)
# 2. Transformacje:
#    - withColumn(), regexp_extract(), length(), filter(), groupBy(), collect_list()
#    Akcja: count()
# 3. Stage’y:
#    - groupBy() + collect_list() może spowodować shuffle → 1 Stage dla narrow + shuffle
# 4. Taski:
#    - Liczba tasków = liczba partycji
#    - Każda partycja = 1 task
# Wnioski:
# - Brak pivot → unikamy błędu [NOT_STR]
# - Brak pełnego Window w jednej partycji → lepsza wydajność



/databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/expressions.py:1134: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


+------+--------+----+-------+
|header|sequence|plus|quality|
+------+--------+----+-------+
+------+--------+----+-------+

Liczba niespójnych rekordów: 0
